In [2]:
import numpy as np
from PIL import Image
from scipy.ndimage import minimum_filter

In [3]:
originalPhoto = Image.open("before_photo.jpg")
before = Image.open("before_photo.jpg")
width, height = before.size

# photo to array
before = np.array(before)
# getting dark channel and kernel filtering
darkChannel = np.min(before, axis = 2)
kernel_size = 15
dark_channel_filtered = minimum_filter(darkChannel, size=kernel_size)

# getting dark channel pixel values
pixelValues = []
for i in range(width):
    for j in range(height):
        pixelValues.append(darkChannel[j][i])

validBrightness = sorted(pixelValues)
validBrightness = validBrightness[len(validBrightness) - 1 - int(0.01 * len(validBrightness)):]

# getting highest intensity pixel indicies in dark channel
brightestDarkChannelPixelIndex = []

for i in range(len(pixelValues)):
    if pixelValues[i] >= validBrightness[0]:
        brightestDarkChannelPixelIndex.append((i % width, i // width))

# getting corresponding rgb intensities of listed indices from the dark channel
brightestPixels = []
for i in brightestDarkChannelPixelIndex:
    r,g,b = originalPhoto.getpixel((i[0], i[1]))
    brightestPixels.append([r,g,b])

# darkChannel = Image.fromarray(darkChannel)
# darkChannel.save("dark channel.png")
# darkChannel.show()

# getting the pixel with the highest r, g and b values
highestR = 0
highestG = 0
highestB = 0
for i in range(len(brightestPixels)):
    if brightestPixels[i][0] > highestR:
        highestR = i
    if brightestPixels[i][1] > highestG:
        highestG = i
    if brightestPixels[i][2] > highestB:
        highestB = i

brightestRPixel = np.reshape(list(originalPhoto.getpixel((highestR % width, highestR // width))), (1,1,3))
brightestGPixel = np.reshape(list(originalPhoto.getpixel((highestG % width, highestG // width))), (1,1,3))
brightestBPixel = np.reshape(list(originalPhoto.getpixel((highestB % width, highestB // width))), (1,1,3))

# getting global atmospheric light
globalAtmosphericLightOptions = [brightestRPixel, brightestGPixel, brightestBPixel]

# creating transmission map[]
transmissionMap = []
for i in range(height):
    temp = []
    for j in range(width):
        temp.append(1 - 0.95 * darkChannel[i][j])
    transmissionMap.append(temp)
transmissionMap = np.array(transmissionMap)
transmissionMap = transmissionMap[:, :, np.newaxis]

sceneRadiance = (before - globalAtmosphericLightOptions[1])/np.maximum(transmissionMap, 0.1) + globalAtmosphericLightOptions[1]

sceneRadiance = Image.fromarray(sceneRadiance, "RGB")
sceneRadiance.save("after.png")
sceneRadiance.show()

In [151]:
# transmissionMap = Image.fromarray(transmissionMap, "L")
# transmissionMap.save("transmission map.png")
# transmissionMap.show()

print(before[200][300])
print(globalAtmosphericLightOptions[0])
after = before - globalAtmosphericLightOptions[0]
print(after[200][300])
# transmissionMap

[181 162 158]
[212, 217, 221]
[-31 -55 -63]
